In [1]:
import requests
import http.client

In [2]:
def verify(rsp):
    try:
        rsp.raise_for_status()
    except Exception as e:
        e.args = e.args[0] + " " + rsp.json()['error'],
        raise e
    print(rsp.json())

In [104]:
url = "http://localhost:8001/api_v0/someendpoint?foo=three&biff=as;ldjf"
auth = "user1","/U5XzhB7CQfo5ZfnVovDQvSL+dG3nF3fwCN2ijJ9cvhXeHwmT2IjuOUZyD1UZJUd"
rsp = requests.get(url, auth=auth)
verify(rsp)

{'Message': 'Success!'}


In [116]:
def check_keys(keys, js):
    for key in keys:
        assert key in js
    for key, _ in js.items():
        assert key in keys
        
def do_req(endpoint, auth, params=None):
    if not endpoint.startswith("/"): endpoint += "/"+endpoint
    rsp = requests.get("http://localhost:8001/api_v0"+endpoint, auth=auth, params=params)
    rsp.raise_for_status()
    return rsp.json()

def test_endpoint(ep, auth, keys, params=None):
    print("Testing endpoint '%s'... "%ep, end="")
    j = do_req(ep, auth, params)
    try:
        check_keys(keys, j)
    except AssertionError as e:
        print("Test Failed: %s"%str(e))
    else:
        print("Success")

def test_infoversions(auth):
    keys = {"SystemSoftware", "DesktopUI", "HelloServer","RIO","Database","Network","DatabaseLogger"}
    test_endpoint("/info/versions", auth, keys)
    
def test_infohardware(auth):
    keys = {"TempSensorConfig", "pHSensorConfig", "DOSensorConfig"}
    test_endpoint("/info/hardware", auth, keys)
    
def test_infosystem(auth):
    keys = {"ReactorName", "Type", "Size", "ComputerName", "ComputerDescription"}
    test_endpoint("/info/system", auth, keys)
    
def _chkky(a,b):
    for key in a:
        assert key in b
    
def test_vector(v, auth):
    js = do_req(v['endpoint'], auth, v['params'])
    assert js == v['ret']
    
# example test vector
example_specs = [
    {
        "endpoint": "/endpoint",
        "params": None,
        "body": None,
        "ret": {
            "key1": "val1",
            "key2": "val2"
        }
    }
]

In [117]:
test_infoversions(auth)
test_infohardware(auth)
test_infosystem(auth)

Testing endpoint '/info/versions'... Success
Testing endpoint '/info/hardware'... Success
Testing endpoint '/info/system'... Success


In [129]:
url2 = "http://localhost:8001/api_v0/controls/system"
rsp=requests.get(url2, auth=auth)
verify(rsp)

HTTPError: 400 Client Error: Bad Request for url: http://localhost:8001/api_v0/controls/system Unknown control type: "system"

In [161]:
def test_controls(auth):
    def keyfix(key):
        return key.lower()
    ctrls = {
        "agitation": {
            "pv": "pv",
            "sp": "auto_sp",
            "man": "man_sp",
            "output": "output"            
        },
        "temperature": {
            "pv": "pv",
            "sp": "auto_sp",
            "man": "man_sp",
            "output": "output"            
        },
        "ph": {
            "pv": "pv",
            "sp": "auto_sp",
            "manUp": "man_base_sp",
            "manDown": "man_co2_sp",
            "outputUp": "base_output",
            "outputDown": "co2_output",
        },
        "do": {
            "pv": "pv",
            "sp": "auto_sp",
            "manUp": "man_o2_sp",
            "manDown": "man_n2_sp",
            "outputUp": "o2_output",
            "outputDown": "n2_output",
        },
        "level": {
            "pv": "pv",
        }, 
        "pressure": {
            "pv": "pv",
        },
        "condenser": {
            "pv": "pv",
            "sp": "auto_sp",
            "man": "man_sp",
            "output": "output"            
        },
        "maingas": {
            "pv": "pv",
            "man": "man_sp"
        },
        "MFCs": {
            "air": "air_flow",
            "co2":"co2_flow",
            "o2":"o2_flow",
            "n2":"n2_flow"
        }
    }
    url1 = "http://localhost:8001/webservice/interface?call=getMainValues&json=1"
    url2 = "http://localhost:8001/api_v0/controls/"
    s = requests.Session()
    mv = s.get(url1).json()
    for ep, values in ctrls.items():
        print("testing %s"%ep)
        vals = s.get(url2+keyfix(ep), auth=auth).json()
        for k, v in values.items():
            exp = mv[ep][k]
            got = vals[v]
            if isinstance(exp, float):
                match = round(exp, 1) == round(got, 1)
            else:
                match = exp == got
            assert match, (ep, k, exp, got)

In [163]:
test_controls(auth)

testing do
testing maingas
testing pressure
testing agitation
testing ph
testing MFCs
testing temperature
testing level
testing condenser


In [165]:
requests.get("http://localhost:8001/api_v0/controls/agitation", auth=auth).json()

{'alarm_status': '',
 'auto_sp': 20,
 'interlocked': False,
 'man_sp': 40,
 'mode': 'auto',
 'output': 35.67894744873047,
 'pv': 20.01471710205078}

In [52]:
import base64, clipboard
clipboard.copy("Basic %s"%base64.b64encode(("%s:%s"%auth).encode()).decode())

In [125]:
s = requests.Session()

url2 = "http://localhost:8001/api_v0/someendpoint"
data = {
    "foo": "3",
    "biff": "4"
}
import time

def update(i, start, last, rsp):
    rsp.raise_for_status()
    now = time.time()
    total = now - start
    diff = now - last
    if i % 10 == 0: print("Request %d completed after %.2f seconds (total %.2f elapsed)" % (i, diff, total))
    return now

unique_urls = set()
iters = 50
start = last = time.time()
for i in range(iters):
    data["foo"] = str(i + 3)
    data["biff"] = str(i + 4)
    req = requests.Request("GET", url=url2, auth=auth, params=data)
    prep = s.prepare_request(req)
    unique_urls.add(prep.path_url)
    rsp = s.get(url2, auth=auth, params=data)
    last = update(i+1, start, last, rsp)
assert len(unique_urls) == iters

Request 10 completed after 0.01 seconds (total 2.13 elapsed)
Request 20 completed after 0.01 seconds (total 2.24 elapsed)
Request 30 completed after 0.01 seconds (total 2.35 elapsed)
Request 40 completed after 0.01 seconds (total 2.48 elapsed)
Request 50 completed after 0.01 seconds (total 2.60 elapsed)
